# sqlite implementationof tournament project
[project description](https://docs.google.com/document/d/1_QQ_FBcPROER-s674YT5WoV6wdpvGWZCI9b8_p0RJ_s/pub)

In [1]:
import sqlite3
conn = sqlite3.connect(":memory:")
c = conn.cursor()

#query = "create database tournament;"  # error, can't create database in api

query = "create table players (id integer primary key autoincrement, name text);"
c.execute(query)

query = "create table matches (matchid integer primary key autoincrement, winner integer references players(id), loser integer references players(id));"
c.execute(query)

### sqlite_master

In [2]:
c.execute("select name from sqlite_master where type = 'table';")
print c.fetchall()

c.execute("select sql from sqlite_master where type = 'table' and name = 'matches';")
print c.fetchall()

[(u'players',), (u'sqlite_sequence',), (u'matches',)]
[(u'CREATE TABLE matches (matchid integer primary key autoincrement, winner integer references players(id), loser integer references players(id))',)]


## 1 testCount()

In [3]:
def deleteMatches():
    c.execute("delete from matches;")
    conn.commit()
deleteMatches()

def deletePlayers():
    c.execute("delete from players;")
    conn.commit()
deletePlayers()    

In [4]:
def countPlayers():
    c.execute("select count(id) from players;")
    return c.fetchone()[0]
num = countPlayers()
if num == '0':
    raise TypeError(
        "countPlayers should return numeric zero, not string '0'.")
if num != 0:
    raise ValueError("After deletion, countPlayers should return zero.")
print "1. countPlayers() returns 0 after initial deletePlayers() execution."

1. countPlayers() returns 0 after initial deletePlayers() execution.


In [5]:
def registerPlayer(name):
    c.execute("insert into players(name) values (?)", (name,))
    conn.commit()
registerPlayer("Chandra Nalaar")

num = countPlayers()
if num != 1:
    raise ValueError(
        "After one player registers, countPlayers() should be 1. Got {c}".format(c=num))
print "2. countPlayers() returns 1 after one player is registered."

registerPlayer("Jace Beleren")
num = countPlayers()
if num != 2:
    raise ValueError(
        "After two players register, countPlayers() should be 2. Got {c}".format(c=num))
print "3. countPlayers() returns 2 after two players are registered."

deletePlayers()
num = countPlayers()
if num != 0:
    raise ValueError(
        "After deletion, countPlayers should return zero.")
print "4. countPlayers() returns zero after registered players are deleted.\n5. Player records successfully deleted."


2. countPlayers() returns 1 after one player is registered.
3. countPlayers() returns 2 after two players are registered.
4. countPlayers() returns zero after registered players are deleted.
5. Player records successfully deleted.


## 2 testStandingsBeforeMatches()

In [6]:
deleteMatches()
deletePlayers()
registerPlayer("Melpomene Murray")
registerPlayer("Randy Schwartz")

In [7]:
def playerStandings():
    """Returns a list of the players and their win records, sorted by wins.
    Returns:
      A list of (id, name, wins, matches):
        id: the player's unique id (assigned by the database)
        name: the player's full name (as registered)
        wins: the number of matches the player has won
        matches: the number of matches the player has played
    """
    c.execute("select id, name, sum(case when id=winner then 1 else 0 end),\
            sum(case when id in (winner,loser) then 1 else 0 end ) \
            from players left join matches \
            group by id  \
            order by count(winner) DESC;")
    return c.fetchall()
standings = playerStandings()

In [8]:
if len(standings) < 2:
    raise ValueError("Players should appear in playerStandings even before "
                     "they have played any matches.")
elif len(standings) > 2:
    raise ValueError("Only registered players should appear in standings.")
if len(standings[0]) != 4:
    raise ValueError("Each playerStandings row should have four columns.")
    
[(id1, name1, wins1, matches1), (id2, name2, wins2, matches2)] = standings
if matches1 != 0 or matches2 != 0 or wins1 != 0 or wins2 != 0:
    raise ValueError(
        "Newly registered players should have no matches or wins.")
if set([name1, name2]) != set(["Melpomene Murray", "Randy Schwartz"]):
    raise ValueError("Registered players' names should appear in standings, "
                     "even if they have no matches played.")
print "6. Newly registered players appear in the standings with no matches."

6. Newly registered players appear in the standings with no matches.


## 3.testReportMatches()


In [9]:
deleteMatches()
deletePlayers()
registerPlayer("Bruno Walton")
registerPlayer("Boots O'Neal")
registerPlayer("Cathy Burton")
registerPlayer("Diane Grant")
standings = playerStandings()
[id1, id2, id3, id4] = [row[0] for row in standings]

In [10]:
def reportMatch(winner, loser):
    """Records the outcome of a single match between two players.
    Args:
      winner:  the id number of the player who won
      loser:  the id number of the player who lost
    """
    c.execute("insert into matches(winner,loser) values (?,?)", (winner,loser))
    conn.commit()

reportMatch(id1, id2)
reportMatch(id3, id4)
standings = playerStandings()
for (i, n, w, m) in standings:
    if m != 1:
        raise ValueError("Each player should have one match recorded.")
    if i in (id1, id3) and w != 1:
        raise ValueError("Each match winner should have one win recorded.")
    elif i in (id2, id4) and w != 0:
        raise ValueError("Each match loser should have zero wins recorded.")
print "7. After a match, players have updated standings."

7. After a match, players have updated standings.


In [11]:
deleteMatches()
standings = playerStandings()
if len(standings) != 4:
    raise ValueError("Match deletion should not change number of players in standings.")
for (i, n, w, m) in standings:
    if m != 0:
        raise ValueError("After deleting matches, players should have zero matches recorded.")
    if w != 0:
        raise ValueError("After deleting matches, players should have zero wins recorded.")
print "8. After match deletion, player standings are properly reset.\n9. Matches are properly deleted."

8. After match deletion, player standings are properly reset.
9. Matches are properly deleted.


## 4 testPairings()

In [12]:
deleteMatches()
deletePlayers()
registerPlayer("Twilight Sparkle")
registerPlayer("Fluttershy")
registerPlayer("Applejack")
registerPlayer("Pinkie Pie")
registerPlayer("Rarity")
registerPlayer("Rainbow Dash")
registerPlayer("Princess Celestia")
registerPlayer("Princess Luna")
standings = playerStandings()
[id1, id2, id3, id4, id5, id6, id7, id8] = [row[0] for row in standings]

In [13]:
def swissPairings():
    """Returns a list of pairs of players for the next round of a match.
    Assuming that there are an even number of players registered, each player
    appears exactly once in the pairings.  Each player is paired with another
    player with an equal or nearly-equal win record, that is, a player adjacent
    to him or her in the standings.

    Returns:
      A list of tuples, each of which contains (id1, name1, id2, name2)
    """
    c.execute("drop view if exists rank;")
    c.execute("create view rank as select id, name, count(winner) as points \
                    from players left join matches \
                    on id=winner \
                    group by id \
                    order by points DESC;")
    c.execute("select a.id, a.name, b.id, b.name from rank as a, rank as b \
                    where a.rowid =b.rowid-1 and (b.rowid %2)=0;")
    results = c.fetchall()
    return results

pairings = swissPairings()
if len(pairings) != 4:
    raise ValueError(
        "For eight players, swissPairings should return 4 pairs. Got {pairs}".format(pairs=len(pairings)))

In [14]:
reportMatch(id1, id2)
reportMatch(id3, id4)
reportMatch(id5, id6)
reportMatch(id7, id8)

pairings = swissPairings()
if len(pairings) != 4:
    raise ValueError(
        "For eight players, swissPairings should return 4 pairs. Got {pairs}".format(pairs=len(pairings)))
[(pid1, pname1, pid2, pname2), (pid3, pname3, pid4, pname4), (pid5, pname5, pid6, pname6), (pid7, pname7, pid8, pname8)] = pairings
possible_pairs = set([frozenset([id1, id3]), frozenset([id1, id5]),
                      frozenset([id1, id7]), frozenset([id3, id5]),
                      frozenset([id3, id7]), frozenset([id5, id7]),
                      frozenset([id2, id4]), frozenset([id2, id6]),
                      frozenset([id2, id8]), frozenset([id4, id6]),
                      frozenset([id4, id8]), frozenset([id6, id8])
                      ])
actual_pairs = set([frozenset([pid1, pid2]), frozenset([pid3, pid4]), frozenset([pid5, pid6]), frozenset([pid7, pid8])])
for pair in actual_pairs:
    if pair not in possible_pairs:
        raise ValueError(
            "After one match, players with one win should be paired.")
print "10. After one match, players with one win are properly paired."
print "Success!  All tests pass!"

10. After one match, players with one win are properly paired.
Success!  All tests pass!


## find some tricks
-  c.description
-  sqlparse

In [15]:
c.execute("select * from players;")
print [i[0] for i in c.description]
c.fetchall()

['id', 'name']


[(9, u'Twilight Sparkle'),
 (10, u'Fluttershy'),
 (11, u'Applejack'),
 (12, u'Pinkie Pie'),
 (13, u'Rarity'),
 (14, u'Rainbow Dash'),
 (15, u'Princess Celestia'),
 (16, u'Princess Luna')]

In [16]:
import sqlparse
query = "create view rank as select id, name, count(winner) as wins from players left join matches \
on id=winner group by id order by wins DESC;"
print sqlparse.format(query,reindent=True)

create view rank as
select id,
       name,
       count(winner) as wins
from players
left join matches on id=winner
group by id
order by wins DESC;
